In [41]:
import os
import numpy as np

##### Loading the Original Data from PEMS07 Dataset

In [43]:
graph_signal_matrix_filename = r"C:\Users\User\Traffic_Flow_Forecasting\ASTGCN_Baseline_Model\PEMS04\PEMS04.npz"
data = np.load(graph_signal_matrix_filename)
data['data'].shape

(16992, 307, 3)

#### The above dataset PEMS07 has 883 traffic sensors recording data with a duration of 3 months 

In [45]:
number_of_days = 16992 / (12 * 24)
print("The required number of days for which the data is collected is ", number_of_days)

The required number of days for which the data is collected is  59.0


##### the 1 hour data(INPUT) is of shape (12, 883, 1) and the next hour(TARGET) is the same (12, 883, 1)

#### The function search_data returns a certain number of time period segment data of duration = 1hr

In [47]:
def search_data(sequence_length, num_of_depend, label_start_idx,num_for_predict, units, points_per_hour):
    '''
    Parameters
    ----------
    sequence_length: int, length of all history data
    num_of_depend: int, number of dependent time periods of length 1 hour
    label_start_idx: int, the first index of predicting target
    num_for_predict: int, the number of points will be predicted for each sample (in case of a hour, for each input sample 12 points to
                                                                                   be predicted)
    units: int, week: 7 * 24, day: 24, recent(hour): 1
    
    points_per_hour: int, number of points per hour, depends on data, here 12 points are considered because 1 hour is considered
    Returns
    ----------
    list[(start_idx, end_idx)]
    '''
    ###Checks if there are non-zero number of points in target sample
    if points_per_hour < 0:
        raise ValueError("points_per_hour should be greater than 0!")

    ###Checking if the selected sequence is bypassing the length of sequence
    if label_start_idx + num_for_predict > sequence_length:
        return None

    x_idx = []
    for i in range(1, num_of_depend + 1):
        start_idx = label_start_idx - points_per_hour * units * i
        end_idx = start_idx + num_for_predict
        if start_idx >= 0:
            x_idx.append((start_idx, end_idx))
        else:
            return None
    ###Checking if we have same number of time periods or not
    if len(x_idx) != num_of_depend:
        return None
    ### return in from of recent time period segmnent order
    return x_idx[::-1]

### The Function 'get_sample_indices' returns the sample data concatenatedwith input (sequence length data), output (target) 

In [49]:
def get_sample_indices(data_sequence, num_of_weeks, num_of_days, num_of_hours, label_start_idx, num_for_predict, points_per_hour=12):
    '''
    Parameters
    ----------
    data_sequence: np.ndarray shape is (sequence_length, num_of_vertices, num_of_features)
    num_of_weeks, num_of_days, num_of_hours: int
    label_start_idx: int, the first index of predicting target
    num_for_predict: int,the number of points will be predicted for each sample
    points_per_hour: int, default 12, number of points per hour
    Returns
    ----------
    week_sample: np.ndarray shape is (num_of_weeks * points_per_hour, num_of_vertices, num_of_features)
    day_sample: np.ndarray shape is (num_of_days * points_per_hour,  num_of_vertices, num_of_features)
    hour_sample: np.ndarray   shape is (num_of_hours * points_per_hour, num_of_vertices, num_of_features)
    target: np.ndarray shape is (num_for_predict, num_of_vertices, num_of_features)
    '''
    week_sample, day_sample, hour_sample = None, None, None
#------------------------------------Ignore
    ###Checks if the length of the predicting segment is outside the sequence lengtrg
    if label_start_idx + num_for_predict > data_sequence.shape[0]: 
        return week_sample, day_sample, hour_sample, None

    if num_of_weeks > 0:
        week_indices = search_data(data_sequence.shape[0], num_of_weeks, label_start_idx, num_for_predict,7 * 24, points_per_hour)
        if not week_indices:
            return None, None, None, None

        week_sample = np.concatenate([data_sequence[i: j] for i, j in week_indices], axis=0)

    if num_of_days > 0:
        day_indices = search_data(data_sequence.shape[0], num_of_days,  label_start_idx, num_for_predict, 24, points_per_hour)
        if not day_indices:
            return None, None, None, None

        day_sample = np.concatenate([data_sequence[i: j] for i, j in day_indices], axis=0)
#----------------------------------Continue

    ###Checks if the number of hours is non-zero
    if num_of_hours > 0:
        hour_indices = search_data(data_sequence.shape[0], num_of_hours, label_start_idx, num_for_predict, 1, points_per_hour)
        if not hour_indices:
            return None, None, None, None

        ###Getting the concatenated data along the sequence length axis
        hour_sample = np.concatenate([data_sequence[i: j] for i, j in hour_indices], axis=0)
    
    if num_of_hours > 10:
        return 1;
    ###Getting the predicting data
    target = data_sequence[label_start_idx: label_start_idx + num_for_predict]

    return week_sample, day_sample, hour_sample, target

In [51]:
def read_and_generate_dataset(graph_signal_matrix_filename, num_of_weeks, num_of_days, num_of_hours, num_for_predict, points_per_hour=12):
    '''
    Parameters
    ----------
    graph_signal_matrix_filename: str, path of graph signal matrix file
    num_of_weeks, num_of_days, num_of_hours: int
    num_for_predict: int
    points_per_hour: int, default 12, depends on data
    Returns
    ----------
    feature: np.ndarray, shape is (num_of_samples, num_of_depend * points_per_hour, num_of_vertices, num_of_features)
    target: np.ndarray, shape is (num_of_samples, num_of_vertices, num_for_predict)
    '''
    #--------------------------------- Read original data 
    data_seq = np.load(graph_signal_matrix_filename)['data']  # (sequence_length, num_of_vertices, num_of_features) (16992, 307, 3)
    
    #---------------------------------
    all_samples = []
    for idx in range(data_seq.shape[0]):
        sample = get_sample_indices(data_seq, num_of_weeks, num_of_days, num_of_hours, idx, num_for_predict, points_per_hour)
        if ((sample[0] is None) and (sample[1] is None) and (sample[2] is None)):
            continue

        week_sample, day_sample, hour_sample, target = sample #  week_sample, day_sample are None because we are predicting per hour
        #print(target.shape) # hour_sample and target (12, 307, 3)
        sample = []  # [(week_sample),(day_sample),(hour_sample),target,time_sample]
#-------------------------------- Ignore
        if num_of_weeks > 0:
            week_sample = np.expand_dims(week_sample, axis=0).transpose((0, 2, 3, 1))  # (1,N,F,T)
            sample.append(week_sample)

        if num_of_days > 0:
            day_sample = np.expand_dims(day_sample, axis=0).transpose((0, 2, 3, 1))  # (1,N,F,T)
            sample.append(day_sample)
#----------------------------------Continue
        if num_of_hours > 0:
            hour_sample = np.expand_dims(hour_sample, axis=0).transpose((0, 2, 3, 1))  # (1,N,F,T)
            sample.append(hour_sample)
        ####Considering only the First Feature
        target = np.expand_dims(target, axis=0).transpose((0, 2, 3, 1))[:, :, 0, :]  # (1,N,T)
        sample.append(target)
        time_sample = np.expand_dims(np.array([idx]), axis=0)  # (1,1)
        sample.append(time_sample)
        all_samples.append(sample)#sampe：[(week_sample),(day_sample),(hour_sample),target,time_sample] = [(1,N,F,Tw),(1,N,F,Td),(1,N,F,Th),(1,N,Tpre),(1,1)]

    split_line1 = int(len(all_samples) * 0.6)
    split_line2 = int(len(all_samples) * 0.8)

    training_set = [np.concatenate(i, axis=0)  for i in zip(*all_samples[:split_line1])] #[(B,N,F,Tw),(B,N,F,Td),(B,N,F,Th),(B,N,Tpre),(B,1)]
    validation_set = [np.concatenate(i, axis=0) for i in zip(*all_samples[split_line1: split_line2])]
    testing_set = [np.concatenate(i, axis=0) for i in zip(*all_samples[split_line2:])]

    return training_set, validation_set, testing_set

In [97]:
data = np.load(graph_signal_matrix_filename)
data['data'].shape

num_of_vertices = 307
points_per_hour = 6
num_for_predict = 6
num_of_weeks = 0
num_of_days = 0
num_of_hours = 1

####Generating the training, test and validation dataset
training_set, validation_set, testing_set = read_and_generate_dataset(graph_signal_matrix_filename, 0, 0, num_of_hours, 
                                                                      num_for_predict, points_per_hour=points_per_hour)

In [99]:
len(training_set)

3

In [101]:
training_set[0].shape

(10185, 307, 3, 6)

In [103]:
training_set[1].shape

(10185, 307, 6)

In [77]:
training_set[2].shape

(10188, 1)

In [79]:
training_set[0][0]

array([[[6.20e+01, 6.10e+01, 7.10e+01, 8.60e+01, 1.03e+02, 1.14e+02],
        [7.70e-03, 7.40e-03, 9.30e-03, 1.12e-02, 1.44e-02, 1.53e-02],
        [6.79e+01, 6.73e+01, 6.84e+01, 6.78e+01, 6.74e+01, 6.76e+01]],

       [[5.60e+01, 4.00e+01, 2.80e+01, 4.70e+01, 3.60e+01, 7.00e+01],
        [1.12e-02, 8.00e-03, 6.80e-03, 8.90e-03, 8.10e-03, 1.42e-02],
        [6.84e+01, 6.86e+01, 6.74e+01, 6.95e+01, 6.81e+01, 6.90e+01]],

       [[9.00e+01, 9.40e+01, 7.90e+01, 9.40e+01, 1.54e+02, 1.57e+02],
        [1.43e-02, 1.56e-02, 1.22e-02, 1.48e-02, 2.58e-02, 2.55e-02],
        [6.82e+01, 6.78e+01, 6.87e+01, 6.89e+01, 6.73e+01, 6.73e+01]],

       ...,

       [[5.60e+01, 5.10e+01, 6.70e+01, 1.26e+02, 2.00e+02, 2.01e+02],
        [9.80e-03, 9.40e-03, 1.17e-02, 2.07e-02, 3.35e-02, 3.46e-02],
        [6.74e+01, 6.60e+01, 6.67e+01, 6.78e+01, 6.79e+01, 6.66e+01]],

       [[4.80e+01, 3.00e+01, 3.10e+01, 6.40e+01, 6.30e+01, 1.07e+02],
        [7.80e-03, 5.00e-03, 5.30e-03, 1.11e-02, 1.06e-02, 1.80e-02],

In [81]:
training_set[0][0].shape

(307, 3, 6)

In [83]:
training_set[1][0]

array([[127., 144., 140., 164., 143., 142.],
       [105., 105., 107., 151., 107., 138.],
       [ 95., 142.,  78., 158., 139.,  93.],
       ...,
       [184., 197., 173., 182., 225., 172.],
       [ 92., 102., 107., 112., 114.,  94.],
       [132., 131., 119., 118., 126., 112.]])

In [85]:
training_set[1][0].shape

(307, 6)

In [87]:
training_set[2][0]

array([6])

### Data Normalization

###### the data are transformed by zero-mean normalization x′ = x −mean(x) to let the average be 0

In [89]:
def normalization(train, val, test):
    '''
    Parameters
    ----------
    train, val, test: np.ndarray (B,N,F,T)
    Returns
    ----------
    stats: dict, two keys: mean and std
    train_norm, val_norm, test_norm: np.ndarray,
                                     shape is the same as original
    '''

    assert train.shape[1:] == val.shape[1:] and val.shape[1:] == test.shape[1:]  # ensure the num of nodes is the same
    mean = train.mean(axis=(0,1,3), keepdims=True)
    std = train.std(axis=(0,1,3), keepdims=True)
    print('mean.shape:',mean.shape)
    print('std.shape:',std.shape)

    def normalize(x):
        return (x - mean) / std

    train_norm = normalize(train)
    val_norm = normalize(val)
    test_norm = normalize(test)

    return {'_mean': mean, '_std': std}, train_norm, val_norm, test_norm

In [91]:
train_x = np.concatenate(training_set[:-2], axis=-1)  # (B,N,F,T')
val_x = np.concatenate(validation_set[:-2], axis=-1)
test_x = np.concatenate(testing_set[:-2], axis=-1)

train_target = training_set[-2]  # (B,N,T)
val_target = validation_set[-2]
test_target = testing_set[-2]

train_timestamp = training_set[-1]  # (B,1)
val_timestamp = validation_set[-1]
test_timestamp = testing_set[-1]

(stats, train_x_norm, val_x_norm, test_x_norm) = normalization(train_x, val_x, test_x)

all_data = {'train': { 'x': train_x_norm, 'target': train_target,'timestamp': train_timestamp},
            'val': {'x': val_x_norm, 'target': val_target, 'timestamp': val_timestamp},
            'test': {'x': test_x_norm, 'target': test_target, 'timestamp': test_timestamp},
            'stats': {'_mean': stats['_mean'], '_std': stats['_std']} }

mean.shape: (1, 1, 3, 1)
std.shape: (1, 1, 3, 1)


In [93]:
print('train x:', all_data['train']['x'].shape)
print('train target:', all_data['train']['target'].shape)
print('train timestamp:', all_data['train']['timestamp'].shape)
print()
print('val x:', all_data['val']['x'].shape)
print('val target:', all_data['val']['target'].shape)
print('val timestamp:', all_data['val']['timestamp'].shape)
print()
print('test x:', all_data['test']['x'].shape)
print('test target:', all_data['test']['target'].shape)
print('test timestamp:', all_data['test']['timestamp'].shape)
print()
print('train data _mean :', all_data['stats']['_mean'].shape, all_data['stats']['_mean'])
print('train data _std :', all_data['stats']['_std'].shape, all_data['stats']['_std'])

train x: (10188, 307, 3, 6)
train target: (10188, 307, 6)
train timestamp: (10188, 1)

val x: (3396, 307, 3, 6)
val target: (3396, 307, 6)
val timestamp: (3396, 1)

test x: (3397, 307, 3, 6)
test target: (3397, 307, 6)
test timestamp: (3397, 1)

train data _mean : (1, 1, 3, 1) [[[[2.07231529e+02]
   [5.13269684e-02]
   [6.34723612e+01]]]]
train data _std : (1, 1, 3, 1) [[[[1.56478211e+02]
   [4.78690919e-02]
   [8.10671516e+00]]]]


In [95]:
file = os.path.basename(graph_signal_matrix_filename).split('.')[0]
dirpath = '.'
filename = os.path.join(dirpath, file + '_r' + str(num_of_hours) + '_d' + str(num_of_days) + '_w' + str(num_of_weeks)) + '_astcgn'
print('save file:', filename)
np.savez_compressed(filename,
                train_x=all_data['train']['x'],train_target=all_data['train']['target'],train_timestamp=all_data['train']['timestamp'],
                val_x=all_data['val']['x'], val_target=all_data['val']['target'],val_timestamp=all_data['val']['timestamp'],
                test_x=all_data['test']['x'], test_target=all_data['test']['target'], test_timestamp=all_data['test']['timestamp'],
                mean=all_data['stats']['_mean'], std=all_data['stats']['_std'])

save file: .\PEMS04_r1_d0_w0_astcgn
